In [ ]:
# Version 1 of EEG imagined speech project
# Turned in as part of senior project
# Used https://www.kaggle.com/vinaykm/eeg-and-dl as starting point
# Uses each line of data as input. Results are not above chance (About 6%)
# Uses EEG dataset: https://zenodo.org/record/3554128#.Ye85WC-cby8 
# April 8, 2022
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

!module load gpu
import os
print(os.listdir("./final_files"))

In [ ]:
load = 1
time_steps = 1000
subsample = 100
#list of csv data files
x_paths = ["training_csvfiles/sub1_data_train.csv", "training_csvfiles/sub2_data_train.csv", 
           "training_csvfiles/sub3_data_train.csv", "training_csvfiles/sub4_data_train.csv", 
           "training_csvfiles/sub5_data_train.csv", "training_csvfiles/sub6_data_train.csv", 
           "training_csvfiles/sub7_data_train.csv", "training_csvfiles/sub8_data_train.csv", 
           "training_csvfiles/sub9_data_train.csv", "training_csvfiles/sub10_data_train.csv",
           "training_csvfiles/sub11_data_train.csv", "training_csvfiles/sub12_data_train.csv", 
           "training_csvfiles/sub13_data_train.csv", "training_csvfiles/sub14_data_train.csv", 
           "training_csvfiles/sub15_data_train.csv", "training_csvfiles/sub16_data_train.csv", 
           "training_csvfiles/sub17_data_train.csv", "training_csvfiles/sub18_data_train.csv", 
           "training_csvfiles/sub19_data_train.csv", "training_csvfiles/sub20_data_train.csv", 
           "training_csvfiles/sub21_data_train.csv"]

#list of csv data files           
y_paths = ["training_csvfiles/sub1_events_train.csv", "training_csvfiles/sub2_events_train.csv", 
           "training_csvfiles/sub3_events_train.csv", "training_csvfiles/sub4_events_train.csv", 
           "training_csvfiles/sub5_events_train.csv", "training_csvfiles/sub6_events_train.csv", 
           "training_csvfiles/sub7_events_train.csv", "training_csvfiles/sub8_events_train.csv", 
           "training_csvfiles/sub9_events_train.csv", "training_csvfiles/sub10_events_train.csv",
           "training_csvfiles/sub11_events_train.csv", "training_csvfiles/sub12_events_train.csv", 
           "training_csvfiles/sub13_events_train.csv", "training_csvfiles/sub14_events_train.csv", 
           "training_csvfiles/sub15_events_train.csv", "training_csvfiles/sub16_events_train.csv", 
           "training_csvfiles/sub17_events_train.csv", "training_csvfiles/sub18_events_train.csv", 
           "training_csvfiles/sub19_events_train.csv", "training_csvfiles/sub20_events_train.csv",
           "training_csvfiles/sub21_events_train.csv"]

In [ ]:
# For each CSV file in x_paths list, append data to x_data
if load:
    x_data = []
    for x_path in x_paths:
        x_path = "./final_files/" + x_path
        with open(x_path) as file:
            for line in file:
                if line[0] == "T": continue
                line_array = []
                for word in range(len(line.split(','))-1):
                    word+=1
                    line_array.append(float(line.split(',')[word]))
                line_array = np.asarray(line_array)
                x_data.append(line_array)
    x_data = np.asarray(x_data)    
        
# For each CSV file in y_paths list, append data to y_data  
    y_data = []
    for y_path in y_paths:
        y_path = "./final_files/" + y_path
        with open(y_path) as file:
            for line in file:
                if line[0] == "T": continue
                line_array = []
                for word in range(len(line.split(','))-1):
                    word+=1
                    line_array.append(int(line.split(',')[word]))
                line_array = np.asarray(line_array)
                y_data.append(line_array)
    y_data = np.asarray(y_data)

print(x_data)
    

In [ ]:
# Layers
model = Sequential()

model.add(Conv2D(filters = 28, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 28, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 28, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 28, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 14, 1)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(14, activation = "relu"))
model.add(BatchNormalization())

model.add(Dense(16, activation = "sigmoid"))

adam = Adam(lr = 0.00001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])

model.summary()

def generator(batch_size):
    while 1:
        
        x_time_data = np.zeros((batch_size, time_steps//subsample, 14))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(x_data)-time_steps)
            x_time_data[i] = x_data[random_index:random_index+time_steps:subsample]
            yy.append(y_data[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

history=model.fit_generator(generator(14), steps_per_epoch = 9000, epochs = 50)

In [ ]:
# Graphs showing training accuracies and loss
import matplotlib.pyplot as plt

def plot_training_history(history):
    acc = history.history['accuracy']
    loss = history.history['loss']
    epochs = range(len(acc))
 
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training acc',linewidth=2)
    plt.title('Training  accuracy')
    plt.legend()
 
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.title('Training loss')
    plt.legend()
 
    plt.show()
plot_training_history(history)

In [ ]:
#list of csv test data files
x_val_paths = ["testing_csvfiles/sub1_data_test.csv", "testing_csvfiles/sub2_data_test.csv", 
           "testing_csvfiles/sub3_data_test.csv", "testing_csvfiles/sub4_data_test.csv", 
           "testing_csvfiles/sub5_data_test.csv", "testing_csvfiles/sub6_data_test.csv", 
           "testing_csvfiles/sub7_data_test.csv", "testing_csvfiles/sub8_data_test.csv", 
           "testing_csvfiles/sub9_data_test.csv", "testing_csvfiles/sub10_data_test.csv",
           "testing_csvfiles/sub11_data_test.csv", "testing_csvfiles/sub12_data_test.csv", 
           "testing_csvfiles/sub13_data_test.csv", "testing_csvfiles/sub14_data_test.csv", 
           "testing_csvfiles/sub15_data_test.csv", "testing_csvfiles/sub16_data_test.csv", 
           "testing_csvfiles/sub17_data_test.csv", "testing_csvfiles/sub18_data_test.csv", 
           "testing_csvfiles/sub19_data_test.csv", "testing_csvfiles/sub20_data_test.csv", 
           "testing_csvfiles/sub21_data_test.csv"]

#list of csv test event files           
y_val_paths = ["training_csvfiles/sub1_events_train.csv", "training_csvfiles/sub2_events_train.csv", 
           "testing_csvfiles/sub3_events_test.csv", "testing_csvfiles/sub4_events_test.csv", 
           "testing_csvfiles/sub5_events_test.csv", "testing_csvfiles/sub6_events_test.csv", 
           "testing_csvfiles/sub7_events_test.csv", "testing_csvfiles/sub8_events_test.csv", 
           "testing_csvfiles/sub9_events_test.csv", "testing_csvfiles/sub10_events_test.csv",
           "testing_csvfiles/sub11_events_test.csv", "testing_csvfiles/sub12_events_test.csv", 
           "testing_csvfiles/sub13_events_test.csv", "testing_csvfiles/sub14_events_test.csv", 
           "testing_csvfiles/sub15_events_test.csv", "testing_csvfiles/sub16_events_test.csv", 
           "testing_csvfiles/sub17_events_test.csv", "testing_csvfiles/sub18_events_test.csv", 
           "testing_csvfiles/sub19_events_test.csv", "testing_csvfiles/sub20_events_test.csv",
           "testing_csvfiles/sub21_events_test.csv"]

In [ ]:
# For each CSV file in x_val_paths list, append data to x_val_data
x_val_data = []
for x_val_path in x_val_paths:
    x_val_path = "./final_files/" + x_val_path    
    with open(x_val_path) as file:
        for line in file:
            if line[0] == "T": continue
            line_array = []
            for word in range(len(line.split(','))-1):
                word+=1
                line_array.append(float(line.split(',')[word]))
            line_array = np.asarray(line_array)
            x_val_data.append(line_array)
x_val_data = np.asarray(x_val_data)    

# For each CSV file in y_val_paths list, append data to y_val_data
y_val_data = []
for y_val_path in y_val_paths:
    y_val_path = "./final_files/" + y_val_path    
    with open(y_val_path) as file:
        for line in file:
            if line[0] == "T": continue
            line_array = []
            for word in range(len(line.split(','))-1):
                word+=1
                line_array.append(int(line.split(',')[word]))
            line_array = np.asarray(line_array)
            y_val_data.append(line_array)
y_val_data = np.asarray(y_val_data)

print(x_val_data)


In [ ]:
# validation data
def val_generator():
    while 1:
        batch_size = 1
        x_time_data = np.zeros((batch_size, time_steps//subsample, 14))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(x_val_data)-time_steps)
            x_time_data[i] = x_val_data[random_index:random_index+time_steps:subsample]
            yy.append(y_val_data[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [ ]:
gen_data = val_generator()
scores = []
num_test = 5000
for i in range(num_test):
    x_test, y_test = next(gen_data)
    while not 1 in y_test:
        x_test, y_test = next(gen_data)
    score=model.evaluate(x_test, y_test, verbose=0)
    scores.append(score[1])
scores = np.asarray(scores)

print("Mean ", np.mean(scores))